In [1]:
import requests
import pandas as pd
today = pd.Timestamp.today().strftime('%B %-d')

In [2]:
headers = {
    'accept': '*/*',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

In [3]:
# Function to determine the winner and margin
def determine_winner(row):
    if row['harris'] > row['trump']:
        winner = "Harris"
        margin = row['harris'] - row['trump']
    else:
        winner = "Trump"
        margin = row['trump'] - row['harris']
    return f"{winner} by {margin:.1f}"

In [4]:
def format_sources(sources):
    # Sort the list alphabetically
    sorted_sources = sorted(sources)
    
    # Handle special case for empty or single-item lists
    if not sorted_sources:
        return ""
    elif len(sorted_sources) == 1:
        return sorted_sources[0]
    
    # Join all but the last item with commas, and the last item with "and"
    return ", ".join(sorted_sources[:-1]) + " and the " + sorted_sources[-1]

---

In [5]:
# Cook 
cook_src = pd.read_csv('https://static.dwcdn.net/data/KTtuN.csv', storage_options=headers, parse_dates=['Date/Time'])[['Date/Time', 'Harris Trend', 'Trump2 Trend']].dropna().rename(columns={'Date/Time': 'date', 'Harris Trend': 'harris', 'Trump2 Trend': 'trump'}).reset_index(drop=True).round(1)

In [6]:
cook_src['date'] = pd.to_datetime(cook_src['date']).dt.strftime('%Y-%m-%d')
cook_src['source'] = 'Cook Political Report'
cook_src['notes'] = ''

In [7]:
cook_latest = cook_src.query('date == date.max()')

In [8]:
# RCP
rcp_src = pd.read_csv('https://stilesdata.com/polling/harris_trump/polls_avg/_trend/harris_trump_trend.csv')[['fetch_date', 'harris_value', 'trump_value']].rename(columns={'trump_value': 'trump', 'harris_value': 'harris', 'fetch_date': 'date'})

In [9]:
rcp_src['source'] = "Real Clear Politics"
rcp_src['notes'] = ''

In [10]:
rcp_latest = rcp_src.query('date == date.max()')

In [11]:
# FiveThirtyEight
fte_src = pd.read_json('https://projects.fivethirtyeight.com/polls/president-general/2024/national/polling-average.json').pivot(columns='candidate', values='pct_estimate', index='date').reset_index().rename(columns={'Trump': 'trump', 'Harris': 'harris', 'Kennedy': 'kennedy'}).round(1)

In [12]:
fte_src['source'] = 'FiveThirtyEight'
fte_src['date'] = pd.to_datetime(fte_src['date']).dt.strftime('%Y-%m-%d')
fte_src['notes'] = 'w/Kennedy'

In [13]:
fte_latest = fte_src.query('date == date.max()').drop('kennedy', axis=1)

In [14]:
# Nate Silver

In [15]:
nate_cols = ['modeldate','state', 'trump','harris', 'rfk']

In [16]:
nate_src = pd.read_csv('https://static.dwcdn.net/data/wB0Zh.csv', storage_options=headers).query('state=="National"').dropna(subset='harris')[nate_cols].rename(columns={'modeldate': 'date', 'rfk': 'kennedy'}).round(1)

In [17]:
nate_src['source'] = 'Silver Bulletin'
nate_src['date'] = pd.to_datetime(nate_src['date'], format='mixed').dt.strftime('%Y-%m-%d')

In [18]:
nate_latest = nate_src.query('date == date.max()').drop('kennedy', axis=1)

In [19]:
# 270toWin

In [20]:
data_dict = requests.get('https://www.270towin.com/polls/php/get-polls-by-state.php?election_year=2024&candidate_name_dem=Harris&candidate_name_rep=Trump&sort_by=date').json()['results']

In [21]:
src_270_cols = ['poll_date_timestamp', 'poll_dem_avg', 'poll_rep_avg']

In [22]:
# Create a DataFrame from the dictionary
src_270 = pd.DataFrame.from_dict(data_dict, orient='index')[src_270_cols].rename(columns={'index': 'state', 'poll_date_timestamp':'date', 'poll_dem_avg': 'harris', 'poll_rep_avg':'trump'}).reset_index()

In [23]:
src_270['date'] = pd.to_datetime(src_270['date'], unit='s').dt.strftime('%Y-%m-%d')

In [24]:
src_270['notes'] = ''
src_270['source'] = '270toWin'

In [25]:
src_270df = src_270.query('index=="0"').drop('index', axis=1).copy()

In [26]:
latest_270 = src_270df.query('date == date.max()')

In [27]:
# Economist

In [28]:
econ_src = pd.read_csv('https://cdn.economistdatateam.com/2024-us-tracker/harris/data/polls/polltracker-latest-trend.csv', storage_options=headers).pivot(columns='candidate_name', index='date', values='pct').reset_index().rename(columns={'Donald Trump': 'trump', 'Kamala Harris': 'harris'}).round(1)

In [29]:
econ_src['source'] = "Economist"
econ_src['notes'] = ""

In [30]:
# All sources

In [31]:
cols = ['date', 'source', 'harris', 'trump']

In [32]:
df = pd.concat([cook_latest, rcp_latest, fte_latest, nate_latest, latest_270, econ_src]).reset_index(drop=True)[cols]

In [33]:
# Apply the function to create the 'winning' column
df['margin'] = df.apply(determine_winner, axis=1)

In [34]:
harris_avg = df['harris'].mean().round(2).astype(float)
trump_avg = df['trump'].mean().round(2).astype(float)

In [35]:
if harris_avg > trump_avg:
    avg_winning = "Vice President Kamala Harris"
    avg_losing = "former President Donald Trump"
    avg_margin = round(harris_avg - trump_avg, 2)
    avg_winning_value = harris_avg
    avg_losing_value = trump_avg
else:
    avg_winning = "Former President Donald Trump"
    avg_losing = "Vice President Kamala Harris"
    avg_margin = round(trump_avg - harris_avg, 2)
    avg_winning_value = trump_avg
    avg_losing_value = harris_avg

In [36]:
# Example list of sources
sources = list(df['source'].unique())

# Format the sources list
formatted_sources = format_sources(sources)

In [37]:
msg = f'{avg_winning} is leading in the national polls to {avg_losing} by a margin of {avg_margin} percentage points, as of {today}, according to an average of six prominent polling averages. The organizations include: {formatted_sources}'
msg

'Vice President Kamala Harris is leading in the national polls to former President Donald Trump by a margin of 1.68 percentage points, as of August 8, according to an average of six prominent polling averages. The organizations include: 270toWin, Cook Political Report, Economist, FiveThirtyEight, Real Clear Politics and the Silver Bulletin'

In [41]:
!jupyter nbconvert --to script --no-prompt --output ../fetch_all_poll_averages fetch_all_poll_averages.ipynb

[NbConvertApp] Converting notebook fetch_all_poll_averages.ipynb to script
[NbConvertApp] Writing 5480 bytes to ../fetch_all_poll_averages.py
